**Do not forget your TODO items**

----------------------

In [39]:
import math
import numpy
import numpy as np
from itertools import chain, tee
import functools
from collections import deque

## Initialize hyperparameters

In [65]:
from us_city import load_us_cities
cities, norm_cities = load_us_cities()

DIM = 2 # dimension of the points
N = cities.shape[0]  # number of cities
M = int(2.5 * N)  # number of intermediate points between the cities

KAPPA_START = 0.2  # starting value of kappa
GAMMA = 1.05  # kappa damping factor

BETA = 10 # inverse temperature

NUM_ITERS = 10000 # number of iterations of the algorithm
WORST_DISTANCE_EPSILON = 0.01

PERTURBATION_RADIUS = 1 # perturbation radius for generating the intermediate points

# TODO: Delete the two parameters below
# Elastic Net parameters
ELASTIC_ALPHA = 0.2
ELASTIC_BETA = 2.0

In [66]:
class ElasticNet(object):
    
    def __init__(self):
        self.kappa = KAPPA_START        
        self.cities = norm_cities

        self.init_intermediate_cities(strategy="centroid")

        self.worst_dists = deque(maxlen=2)


    def init_intermediate_cities(self, strategy="centroid", **kwargs):
        if strategy == "centroid":
            self.intermediate_cities = np.tile(
                np.mean(self.cities, axis=0),
                M
            ).reshape((M, DIM))
            # Create perturbation vectors of magnitude radius
            perturbation_vectors = np.random.random((M, DIM))
            perturbation_vectors /= np.linalg.norm(perturbation_vectors, axis=1, ord=2, keepdims=True)
            perturbation_vectors *= PERTURBATION_RADIUS
            self.intermediate_cities += perturbation_vectors
        else:
            raise RuntimeError(f"Unknown strategy: {strategy}")


    def solve(self):
        for i in range(NUM_ITERS):
            self.update_weights()
            self.update_kappa()
            self.update_intermediate_cities()
            if i % 100 == 0:
                print(f"Iteration {i}, Worst distance: {self.worst_dist}")
            if self.worst_dist < WORST_DISTANCE_EPSILON:
                break
            if len(self.worst_dists) == 2:
                if self.worst_dists[1] - self.worst_dists[0] == 0:
                    break


    def update_kappa(self):
        self.kappa = max(self.kappa / GAMMA, 0.01)


    def update_weights_2(self):
        def softmax(x):
            """Compute softmax values for each sets of scores in x."""
            e_x = np.exp(x - np.max(x))
            return e_x / e_x.sum()
        # Calculate distances between the cities and the intermediate_cities
        self.diff = self.cities[:, np.newaxis] - self.intermediate_cities
        self.dist_squared = np.sum(self.diff ** 2, axis=-1)

        # self.weights = np.exp(-self.dist_squared / (2 * (self._k ** 2)))
        # self.weights /= self._weights.sum(axis=1)[:,numpy.newaxis]

        self.weights = softmax(BETA * np.sqrt(self.dist_squared))


    def update_weights(self):
        """Compute w_ij, i = 1, 2, ..., |Cities|; j = 1, 2, ...., |intermediate_cities|"""
        
        self.diff = self.cities[:,np.newaxis] - self.intermediate_cities
        self.dist_squared = np.sum(self.diff ** 2, axis=-1)

        self.worst_dist = np.max(np.min(np.sqrt(self.dist_squared), axis=1))
        self.worst_dists.append(self.worst_dist)

        self.weights = np.exp(-self.dist_squared / (2 * (self.kappa ** 2)))
        self.weights /= self.weights.sum(axis=1)[:,np.newaxis]


    def get_D(self):
        def d_ij(i, j):            
            if i == j:
                return np.sum(self.weights[:, int(j)])
            return 0
        return np.fromfunction(np.vectorize(d_ij), (M, M))


    @functools.cached_property
    def L(self):
        def L_ij(i, j, M):
            if i == j:
                return 2
            if i == (j + 1) % M:
                return 1
            if j == (i + 1) % M:
                return 1
            return 0
        return np.fromfunction(np.vectorize(L_ij), (M, M), M=M, dtype=int)


    def update_intermediate_cities(self):
        self.intermediate_cities = np.linalg.inv((self.kappa * self.L + self.get_D())) @ self.weights.T @ self.cities

In [67]:
net = ElasticNet()
net.solve()

Iteration 0, Worst distance: 1.5097365141615824


In [68]:
net.worst_dist

0.0915134133738775

In [69]:
net.weights

array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 7.72424792e-019],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        7.52689873e-230, 0.00000000e+000, 0.00000000e+000],
       ...,
       [0.00000000e+000, 4.41625020e-159, 9.97409047e-203, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 1.00787187e-304],
       [0.00000000e+000, 4.07026548e-105, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 1.32458316e-300]])

In [71]:
net.intermediate_cities

array([[-3.25912853e-01, -8.04847055e-02],
       [ 1.50684417e-01, -1.54829043e-01],
       [ 2.45440194e-02,  3.90142792e-01],
       [-1.23820879e-02, -6.19597368e-01],
       [ 2.20156346e-04,  8.49051944e-01],
       [-1.00738400e-02, -6.17847221e-01],
       [ 1.99275236e-02,  3.86642498e-01],
       [ 3.00336378e-01, -3.99212960e-03],
       [-3.85641508e-01, -1.98939161e-01],
       [ 4.70946639e-01,  4.01870451e-01],
       [-4.28693423e-01, -5.47944421e-01],
       [ 3.86440207e-01,  6.94018390e-01],
       [-2.21684488e-01, -7.84883478e-01],
       [ 5.69287694e-02,  8.75748566e-01],
       [-2.94944893e-02, -7.36592249e-01],
       [ 2.06020919e-03,  5.97435933e-01],
       [-1.80646848e-01, -4.45775330e-01],
       [ 3.59233488e-01,  2.94114728e-01],
       [-4.43620957e-01, -2.14286653e-01],
       [ 5.28008427e-01,  1.34458578e-01],
       [-6.35460809e-01, -8.58541741e-02],
       [ 7.42913190e-01,  3.72497700e-02],
       [-5.24954405e-01, -1.91533852e-02],
       [ 3.

In [72]:
def decode_solution(dist2):
    """Return the permutation associated to the elastic."""

    neuron_city_pair = []

    for _ in range(dist2.shape[0]):
        # Find the city 
        city = dist2.min(axis=1).argmin()
        neuron = dist2[city].argmin()

        dist2[city] = numpy.inf
        dist2[:, neuron] = numpy.inf

        neuron_city_pair.append((neuron, city))

    neuron_city_pair.sort(key=lambda x: x[0])
    return [x[1] for x in neuron_city_pair]


def compute_solution(permutation):
    pass    


def _pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


def compute_solution(permutation, original_cities):
    edges = list(_pairwise(chain(permutation, [permutation[0]])))

    length = sum([distance(original_cities[src_dst[0]],
                           original_cities[src_dst[1]]) for src_dst in edges])

    return (length, edges)


def distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return int(math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) + 0.5)


In [73]:
permutation = decode_solution(net.dist_squared)
permutation

[13, 4, 0, 8, 3, 11, 16, 17, 18, 21, 19, 20, 15, 12, 7, 1, 2, 5, 6, 10, 9, 14]

In [75]:
compute_solution(permutation, cities)

(477,
 [(13, 4),
  (4, 0),
  (0, 8),
  (8, 3),
  (3, 11),
  (11, 16),
  (16, 17),
  (17, 18),
  (18, 21),
  (21, 19),
  (19, 20),
  (20, 15),
  (15, 12),
  (12, 7),
  (7, 1),
  (1, 2),
  (2, 5),
  (5, 6),
  (6, 10),
  (10, 9),
  (9, 14),
  (14, 13)])